대여소 ID, 구 단위 저장


In [46]:
import numpy as np
import pandas as pd


data= pd.read_csv('./따릉이/서울시 따릉이대여소 마스터 정보.csv',encoding='cp949')

대여소_data = data[['대여소_ID', '주소1']]

대여소_data['주소1'] = data['주소1'].str.extract(r'(\S+)구')

강남_대여소_ID = 대여소_data.loc[대여소_data['주소1'] == '강남', '대여소_ID']
강남_대여소_ID_df = pd.DataFrame({'대여소_ID': 강남_대여소_ID})

print(강남_대여소_ID)

36       ST-966
39       ST-963
40       ST-962
41       ST-961
42       ST-960
         ...   
3090    ST-1174
3091    ST-1173
3092    ST-1172
3093    ST-1171
3094    ST-1170
Name: 대여소_ID, Length: 191, dtype: object


C:\Users\ciw96\AppData\Local\Temp\ipykernel_51184\2551918976.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  대여소_data['주소1'] = data['주소1'].str.extract(r'(\S+)구')


따릉이 데이터

In [20]:
import numpy as np
import pandas as pd

data2 = pd.read_csv('./따릉이/tpss_bcycl_od_statnhm_20240423.csv', encoding='cp949')

data2 = data2[data2['집계_기준'] != '도착시간']

따릉이_data = data2[['기준_날짜', '기준_시간대', '시작_대여소_ID', '종료_대여소_ID', '전체_건수']]
따릉이_data['기준_시간대'] = 따릉이_data['기준_시간대'].astype(str).apply(lambda x: f"{int(x)//100}시 {int(x)%100}분")


# 출발 대여소 ID가 강남구 대여소 ID에 해당하는지 확인하는 조건 추가
강남출발_mask = 따릉이_data['시작_대여소_ID'].isin(강남_대여소_ID)

# 도착 대여소 ID가 강남구 대여소 ID에 해당하는지 확인하는 조건 추가
강남도착_mask = 따릉이_data['종료_대여소_ID'].isin(강남_대여소_ID)

# 출발 혹은 도착 대여소 ID가 강남구 대여소 ID에 해당하는 데이터만 남기기
강남_따릉이_data = 따릉이_data[강남출발_mask | 강남도착_mask]

print(강남_따릉이_data)


           기준_날짜   기준_시간대 시작_대여소_ID 종료_대여소_ID  전체_건수
34      20240423    0시 0분   ST-1365   ST-3188      1
53      20240423    0시 0분   ST-1578   ST-2796      1
75      20240423    0시 0분    ST-187    ST-823      1
76      20240423    0시 0분   ST-1888   ST-1185      1
77      20240423    0시 0분   ST-1893   ST-1560      1
...          ...      ...       ...       ...    ...
296479  20240423  23시 50분   ST-1888   ST-1364      1
296480  20240423  23시 50분   ST-1922    ST-813      1
296542  20240423  23시 50분    ST-815   ST-1247      1
296543  20240423  23시 50분    ST-823    ST-823      1
296808  20240423  23시 55분   ST-1407   ST-1364      1

[6933 rows x 5 columns]


C:\Users\ciw96\AppData\Local\Temp\ipykernel_51184\4124904032.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  따릉이_data['기준_시간대'] = 따릉이_data['기준_시간대'].astype(str).apply(lambda x: f"{int(x)//100}시 {int(x)%100}분")


굳이 할 필요 없던거.... 
강남구 하루 이용 횟수 출발, 도착

In [21]:

강남_출발_num = 0
강남_도착_num = 0

for i,따릉이 in 강남_따릉이_data.iterrows():
    시작_대여소_ID = 따릉이['시작_대여소_ID']
    종료_대여소_ID = 따릉이['종료_대여소_ID']
  
    if 시작_대여소_ID in 강남_대여소_ID_df.values:
        강남_출발_num += 따릉이['전체_건수']
    
    if 종료_대여소_ID in 강남_대여소_ID_df.values:
        강남_도착_num += 따릉이['전체_건수']


print("강남에서 출발한 대여 건수:", 강남_출발_num)
print("강남으로 도착한 대여 건수:", 강남_도착_num)

강남에서 출발한 대여 건수: 5666
강남으로 도착한 대여 건수: 5476


기상청 데이터

기온 : 평균 기온? 최고 최저는 편차가 심하면 못 쓸 가능성 높음


In [43]:
import numpy as np
import pandas as pd

def convert_date_to_numeric(date):
    date_str = str(date).replace('-', '')
    return int(date_str)

data3 = pd.read_csv('./기상청/기상청.csv',encoding='cp949',skiprows=8)

열이름 = ['기준_날짜','평균_기온','최고_기온']
기온 = data3.iloc[:,1:4]
기온.columns = 열이름
기온['최저_기온'] = data3.iloc[:,5]
기온['기준_날짜'] = 기온['기준_날짜'].apply(convert_date_to_numeric)
print(기온)


        기준_날짜  평균_기온  최고_기온  최저_기온
108  20230101   -0.2    3.8   -4.3
108  20230102   -4.5   -0.4   -7.4
108  20230103   -5.0    0.6   -9.0
108  20230104   -1.8    3.3   -5.7
108  20230105   -1.6    3.6   -5.6
..        ...    ...    ...    ...
108  20231227    1.5    6.8   -2.8
108  20231228    1.2    4.4   -1.7
108  20231229    1.3    4.6   -1.5
108  20231230    0.6    2.1   -0.9
108  20231231    2.1    4.3    0.6

[365 rows x 4 columns]


강수량 : 

In [44]:
import numpy as np
import pandas as pd

def convert_date_to_numeric(date):
    date_str = str(date).replace('-', '')
    return int(date_str)


data4 = pd.read_csv('./기상청/강수량.csv',encoding='cp949',skiprows=6)
data4 = data4.dropna()

강수량 = data4[['날짜', '강수량(mm)']]
강수량 = 강수량.rename(columns={'날짜': '기준_날짜'})
강수량 = 강수량.rename(columns={'강수량(mm)': '강수량'})
강수량['기준_날짜'] = 강수량['기준_날짜'].apply(convert_date_to_numeric)

print(강수량)

        기준_날짜   강수량
5    20230106   3.9
6    20230107   0.1
11   20230112   0.0
12   20230113  37.3
13   20230114   1.6
..        ...   ...
353  20231220   0.8
357  20231224   1.1
358  20231225   0.7
363  20231230  13.1
364  20231231   4.7

[150 rows x 2 columns]


미세먼지

In [45]:
import numpy as np
import pandas as pd

data5 = pd.read_csv('./기상청/미세먼지.csv',encoding='cp949')

미세먼지_강남구_data = data5[data5['측정소명']=='강남구']
미세먼지_강남구_data = 미세먼지_강남구_data.rename(columns={'측정일시': '기준_날짜'})
미세먼지 = 미세먼지_강남구_data[['기준_날짜','미세먼지(㎍/㎥)']]

print('미세먼지')
print(미세먼지)

미세먼지
         기준_날짜  미세먼지(㎍/㎥)
0     20230101       52.0
25    20230102       24.0
50    20230103       29.0
75    20230104       35.0
100   20230105       47.0
...        ...        ...
9000  20231227       69.0
9025  20231228       48.0
9050  20231229       44.0
9075  20231230       42.0
9100  20231231       23.0

[365 rows x 2 columns]
